<a href="https://colab.research.google.com/github/Bosco77/03MAIR---Algoritmos-de-Optimizacion---2019/blob/master/AG3/AlexVilalta_AG3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Actividad Guiada 3    
Nombre: Alex Vilalta   
Colab: https://colab.research.google.com/drive/1IpdchRyQALOaz_od210wcsEHIsEfqxnv   
Github: 

In [0]:
import urllib.request

file = "swiss42.tsp"

urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp", file)

('swiss42.tsp', <http.client.HTTPMessage at 0x7f0f4d403b70>)

In [0]:
!pip install tsplib95



In [0]:
import tsplib95
import random
import math

In [0]:
problem = tsplib95.load_problem(file)
#NODOS 
Nodos = list(problem.get_nodes())

#Aristas
Aristas = list(problem.get_edges())

In [0]:
#Se genera una solucion aleatoria con comienzo en en el nodo 0
def crear_solucion(Nodos): 
  solucion = [0]
  for i in range(len(Nodos)-1):
    solucion = solucion + [random.choice(list(set(Nodos) - set(solucion)))]
  return solucion  


#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
  return problem.wfunc(a,b)


#Devuelve la distancia total de una trayectoria
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)

In [0]:

#Busqueda aleatorio
def busqueda_aleatoria(problem, N):
  mejor_solucion = crear_solucion(Nodos)
  mejor_valor    = distancia_total(mejor_solucion, problem)

  for i in range(N):
      solucion = crear_solucion(Nodos)
      valor    = distancia_total(solucion, problem)
      if valor < mejor_valor:
        mejor_solucion = solucion
        mejor_valor = valor

  print("La mejor solucion es ",mejor_valor,  mejor_solucion)
   
busqueda_aleatoria(problem, 20000)

La mejor solucion es  3663 [0, 31, 17, 5, 2, 1, 3, 10, 39, 15, 23, 41, 28, 19, 12, 37, 16, 6, 11, 29, 4, 35, 36, 14, 30, 24, 40, 21, 22, 9, 20, 34, 38, 13, 7, 25, 18, 27, 26, 8, 32, 33]


In [0]:
def genera_vecina(solucion):
  #Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
  #print(solucion)
  mejor_solucion = []
  mejor_distancia = 10e100
  for i in range(1,len(solucion)-1):
    for j in range(i+1, len(solucion)):
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:] 
      distancia_vecina = distancia_total(vecina, problem)
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  print(mejor_solucion)
  print(mejor_distancia)
  return mejor_solucion
      
solucion = crear_solucion(Nodos)
mejor_solucion = genera_vecina(solucion)

[0, 21, 32, 1, 18, 28, 15, 4, 39, 13, 35, 26, 11, 10, 23, 30, 41, 38, 8, 36, 27, 16, 14, 9, 19, 31, 33, 12, 5, 2, 29, 7, 6, 22, 24, 25, 40, 37, 3, 17, 34, 20]
4621


In [0]:
def busqueda_local(problem, N):
  #problem = datos del problema
  #N = numero de iteraciones
  
  mejor_solucion = []
  mejor_distancia = 10e100
  
  solucion_referencia = crear_solucion(Nodos)
  
  for i in range(N):
    vecina = genera_vecina(solucion_referencia)
    distancia_vecina = distancia_total(vecina, problem)
    if distancia_vecina <= mejor_distancia:
      mejor_solucion = vecina
      mejor_distancia = distancia_vecina
        
    solucion_referencia = vecina

  print("La mejor solución encontrada es " , mejor_distancia , mejor_solucion)



In [0]:
def genera_vecina_aleatorio(solucion):
  #Generador de 1 solucion vecina 2-opt (intercambiar 2 nodos)
  #Se puede mejorar haciendo que la elección no se uniforme sino entre las que estén más proximas
  i = random.choice(range(1, len(solucion)) )
  j = random.choice(list(set(range(1, len(solucion))) - {i}))
  if j<i:
    i,j=j,i
  vecina = list(solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:])
  return vecina

In [0]:
#Funcion de probabilidad para determinar si se cambia 
#    a una solución peor respecto a la de referencia(exponencial)
def probabilidad(T,d):
  if random.random() <= math.exp(-1*d / T)   :
    return True
  else:
    return False


def bajar_temperatura(T):
  return T*.9999

In [0]:
#####################################################################
def recocido_simulado(problem, TEMPERATURA=1000):
  #problem = datos del problema

  solucion_referencia = crear_solucion(Nodos)
  distancia_referencia = distancia_total(solucion_referencia, problem)
  
  mejor_solucion = []
  mejor_distancia = 10e100
  while TEMPERATURA > 1:
    #Genera una solución vecina(aleatoria)
    vecina = genera_vecina_aleatorio(solucion_referencia) 
    
    #Calcula su valor(distancia)
    distancia_vecina = distancia_total(vecina, problem)
    
    #Si es la mejor solución de todas se guarda
    if distancia_vecina < mejor_distancia:
      mejor_solucion = vecina
      mejor_distancia = distancia_vecina
      print('\t****Encontrada mejor solucion:', mejor_distancia , mejor_solucion, '\n')
    
    #Si la nueva vecina es mejor se cambia y si es peor se cambia según 
    # una probabilidad dependiente de T y de |distancia_referencia - distancia_vecina|
    p = probabilidad(TEMPERATURA, abs(distancia_referencia - distancia_vecina) )
    if distancia_vecina < distancia_referencia or p :
      solucion_referencia = vecina
      distancia_referencia = distancia_vecina
    
    TEMPERATURA = bajar_temperatura(TEMPERATURA)

  print("La mejor solución encontrada es :" , mejor_distancia, mejor_solucion  )
  return mejor_solucion

sol = recocido_simulado(problem, TEMPERATURA=100000)

In [0]:
def Add_Nodo(problem, H ,T ) :
  #Establecer una una funcion de probabilidad para 
  # añadir un nuevo nodo dependiendo de los nodos mas cercanos y de las feromonas depositadas
  Nodos = list(problem.get_nodes())
  return random.choice(   list(set(range(1,len(Nodos))) - set(H) )  )


def Incrementa_Feromona(problem, T, H):
  #Incrementar segun la calidad de la solución. Añadir una cantidad inversamente proporcional a la distancia total 
  for i in range(len(H)-1):
    T[H[i]][H[i+1]] += 1000/distancia_total(H, problem)
  return T

def Evaporar_Feromonas(T):
  #Podemos elegir diferentes funciones de evaporación dependiendo de la cantidad actual y de la suma total de feromonas depositadas,...
  #Evapora 0.3 el valor de la feromona, sin que baje de 1
  T = [[ max(T[i][j] - 0.3 , 1) for i in range(len(Nodos)) ] for j in range(len(Nodos))]
  return T


def hormigas(problem, N) :
  #problem = datos del problema
  #N = Número de agentes(hormigas)
    
  #Nodos
  Nodos = list(problem.get_nodes())
    #Aristas
  Aristas = list(problem.get_edges()) 
  
  #Inicializa las aristas con una cantidad inicial de feromonas:1
  T = [[ 1 for _ in range(len(Nodos)) ] for _ in range(len(Nodos))]
  
  #Se generan los agentes(hormigas) que serán estructuras de caminos desde 0
  Hormiga = [[0] for _ in range(N)]

  #Recorre cada agente construyendo la solución
  for h in range(N) :
    #print("\nAgente:", h)
    #Para cada agente se construye un camino
    for i in range(len(Nodos)-1) :
      
      #Elige el siguiente nodo
      Nuevo_Nodo = Add_Nodo(problem, Hormiga[h] ,T )
      
      Hormiga[h].append(Nuevo_Nodo)     
    
    #Incrementa feromonas en esa arista 
    T = Incrementa_Feromona(problem, T, Hormiga[h] )
    #print("Feromonas(1)", T)
      
    #Evapora Feromonas  
    T = Evaporar_Feromonas(T)
    #print("Feromonas(2)", T)

    #Seleccionamos el mejor agente
  mejor_solucion = []
  mejor_distancia = 10e100
  for h in range(N) :
    distancia_actual = distancia_total(Hormiga[h], problem)
    if distancia_actual < mejor_distancia:
      mejor_solucion = Hormiga[h]
      mejor_distancia =distancia_actual
  
  
  print(mejor_solucion)
  print(mejor_distancia)
  
hormigas(problem, N=1000)

[0, 33, 36, 17, 4, 12, 19, 27, 7, 31, 30, 8, 5, 13, 40, 10, 23, 41, 9, 29, 18, 26, 28, 1, 3, 2, 14, 35, 32, 20, 39, 38, 16, 15, 37, 25, 11, 6, 22, 34, 24, 21]
3601
